In [1]:
import os
import numpy as np
import importlib
import pandas as pd

os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # This is to disable GPU
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

import prepare_ml_dataset
import train_nn_model
import initialize_var

importlib.reload(train_nn_model)
importlib.reload(prepare_ml_dataset)
importlib.reload(initialize_var)

# For Ring Current, we will use energy  bins from 1 keV to 50 keV
# ENERGY_BINS = (np.array([51767.680, 44428.696, 38130.120, 32724.498, 28085.268, 24103.668
# , 20686.558, 17753.876, 15236.896, 13076.798, 11222.936, 9631.899, 8266.406, 7094.516, 6088.722, 5225.528, 4484.742
# , 3848.919, 3303.284, 2834.964, 2433.055, 2088.129, 1792.096, 1538.062, 1319.977, 1132.846, 972.237]))

/home/jliao/anaconda3/envs/notebook/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-25 11:04:08.031494: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-25 11:04:08.115533: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-25 11:04:08.195946: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742915048.251503  142659 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742915048.267852  142659 cuda_blas.c

<module 'initialize_var' from '/home/jliao/workspace/GitHub/ml_ringcurrent_ion/initialize_var.py'>

In [ ]:
importlib.reload(train_nn_model)

energy = '51767680'
species = 'h'
forecast = "none"
dL01=True
save_data = False
plot_data = False
recalc = False
number_history = 7
raw_feature_names=['symh','asyh','ae','asyd']
forecast = "none"

train_nn_model.train_nn_model(energy, species, recalc = recalc, plot_data = plot_data, save_data = save_data, dL01=dL01
                              , number_history = number_history, raw_feature_names = raw_feature_names, forecast = forecast)

In [ ]:

energy = '51767680'
species = 'h'
forecast = "none"
dL01=True
save_data = False
plot_data = False
recalc = False
number_history = 7
raw_feature_names=['symh','asyh','ae','asyd']
forecast = "none"

np.set_printoptions(precision=4)

dataset_csv, data_settings, directories = initialize_var.initialize_data_var(energy=energy, species=species, raw_feature_names = raw_feature_names, forecast = forecast, number_history = number_history, dL01=dL01)

x_train, x_valid, x_test, y_train, y_valid, y_test = prepare_ml_dataset.prepare_ml_dataset(energy, species, recalc = recalc, plot_data = plot_data, save_data = save_data, dL01=dL01
                                                                                           , raw_feature_names = raw_feature_names,  forecast = forecast, number_history = number_history)

In [ ]:
x_train

In [ ]:
importlib.reload(train_nn_model)

para_name = "learning_rate"
para_set = [1.e-4, 1.5e-3, 1.e-3]

final_train_loss = np.zeros(len(para_set))
final_valid_loss = np.zeros(len(para_set))
total_history = dict()
valid_r2s = np.zeros(len(para_set))

ipara = 0

parameter = para_set[ipara]

model, history, valid_r2 = train_nn_model.nn_model(x_train, y_train, x_valid, y_valid, data_settings["log_y_name"], output_dir = directories["training_output_dir"] , model_fln = '', mse_fln = ''
                                    , n_neurons = 18, dropout_rate = 0.0, patience = 32, learning_rate = parameter, epochs = 2, batch_size = 8, dL01= dL01)

total_history[str(parameter)] = history.history
final_train_loss[ipara] = history.history['loss'][-1]
final_valid_loss[ipara] = history.history['val_loss'][-1]
valid_r2s[ipara] = valid_r2

